In [143]:
from transformers import BertModel, BertTokenizer, logging
import torch
import numpy as np
import re
import glob
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from datasets import load_dataset
logging.set_verbosity_error()

In [27]:
pos_path = os.path.dirname(mycwd)+ '/data/aclImdb/train/pos/'
neg_path = os.path.dirname(mycwd)+ '/data/aclImdb/train/neg/'
#pos_path = 'E:/MLFun/data/aclImdb_v1/aclImdb/train/pos/'
#neg_path = 'E:/MLFun/data/aclImdb_v1/aclImdb/train/neg/'

In [34]:
with open(pos_path + '0_9.txt') as f:
    print(f.readlines()[0])

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


In [50]:
# cleans all the <br>'s for now. TODO: clean '\' as well
def clean(text):
    ret = re.sub(r"\<br..\>", ' ', text)
    return ret

# get all txt files from given path and append appropriate training value
def get_data(data_path, val):
    ret = []
    files = glob.glob(data_path + "*.txt")
    for fle in files:
        with open(fle, encoding="utf8") as f:
            text = f.readlines()[0]
            text = clean(text)
            ret.append([text,val])
    return ret

In [51]:
X_train = []
X_train += get_data(pos_path, 1) + get_data(neg_path, 0)
X_train = np.array(X_train)

In [121]:
np.random.shuffle(X_train)
temp = X_train[0:1000]

In [122]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [123]:
def embed_texts(x, model, tokenizer):
    """
    Create embedding vectors for the input articles 
    
    Args:
        articles (list(dict)): 
            A list of articles 
    
    Returns: 
        (numpy.ndarray or other format):
            An MxN matrix/table for the article embeddings, where 
            each row is for the embedding vector for an article
    """
    text  = x
    #tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    #model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
    #model.eval()
    sentence_emb = torch.LongTensor(tokenizer.encode(text, truncation = True))
    sentence_emb = sentence_emb.to(device)
    with torch.no_grad():
        # embed the sentences
        out = model(sentence_emb.unsqueeze(0))
        hidden_states = out[2]
            
    # sum up last four layers for improved performance
    last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
        
    # reshape the embedding to (1,768)
    sum_sentence_embedding = torch.mean(sum(last_four_layers), dim=1).squeeze()
    ret = np.array(sum_sentence_embedding.cpu())
    
    return ret

In [59]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
model = model.to(device)

In [64]:
sentence_emb = torch.LongTensor(tokenizer.encode(temp[0][0], truncation = True))
sentence_emb = sentence_emb.to(device)
with torch.no_grad():
    # embed the sentences
    out = model(sentence_emb.unsqueeze(0))
    hidden_states = out[2]
last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
        
# reshape the embedding to (1,768)
sum_sentence_embedding = torch.mean(sum(last_four_layers), dim=1).squeeze()
ret = np.array(sum_sentence_embedding.cpu())
#ret

In [124]:
model.eval()
df = pd.DataFrame(temp)
df.rename(columns={0: 'text', 1: 'label'}, inplace=True)

In [125]:
df['embedding'] = df.apply(lambda x: embed_texts(x['text'], model, tokenizer), axis = 1)

In [126]:
X = np.vstack(df['embedding'].to_numpy())
y = df['label'].to_numpy()
y

array(['1', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '0', '1',
       '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0',
       '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '0', '1',
       '0', '1', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0',
       '0', '1', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '0',
       '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1',
       '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0',
       '1', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '1',
       '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '0', '0', '0',
       '1', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0',
       '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0',
       '1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0',
       '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '0

In [137]:
pca = PCA(200)
X_pca = pca.fit_transform(X)
sum(pca.explained_variance_ratio_)

0.941831381409429

In [138]:
clf = LogisticRegression(random_state=0).fit(X_pca, y)

In [141]:
clf.predict([embed_texts(X_train[1][0], model, tokenizer)])

ValueError: X has 768 features, but LogisticRegression is expecting 200 features as input.

In [140]:
clf.score(X_pca, y)

0.942